In [1]:
from typing import List, Dict, Any
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_core.tools import BaseTool
from langchain.tools import Tool
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
import json
from langchain_openai import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory

In [2]:
import os 
from dotenv import load_dotenv
load_dotenv('../../.env')

model=os.environ.get('model')
print(model)

os.environ["OPENAI_API_VERSION"] = os.environ.get('AZURE_API_VERSION')
os.environ["AZURE_OPENAI_ENDPOINT"] = os.environ.get('AZURE_API_BASE')
os.environ["AZURE_OPENAI_API_KEY"] = os.environ.get('AZURE_API_KEY')

azure/gpt-4


In [3]:
class TravelTools:
    @staticmethod
    def search_flights():
        return Tool(
            name="search_flights",
            func=lambda x: "Final Answer: Flight search results for: " + x,
            description="Search for available flights"
        )

    @staticmethod
    def search_trains():
        return Tool(
            name="search_trains",
            func=lambda x: "Final Answer: Train search results for: " + x,
            description="Search for available trains"
        )

    @staticmethod
    def search_hotels():
        return Tool(
            name="search_hotels",
            func=lambda x: "Final Answer: Hotel search results for: " + x,
            description="Search for available hotels"
        )

class FinanceTools:
    @staticmethod
    def search_stocks():
        return Tool(
            name="search_stocks",
            func=lambda x: "Final Answer: Stock information for: " + x,
            description="Get stock market information"
        )

    @staticmethod
    def search_mutual_funds():
        return Tool(
            name="search_mutual_funds",
            func=lambda x: "Final Answer: Mutual fund information for: " + x,
            description="Get mutual fund information"
        )

    @staticmethod
    def get_financial_advice():
        return Tool(
            name="get_financial_advice",
            func=lambda x: "Final Answer: Financial advice for: " + x,
            description="Get general financial advice"
        )


In [4]:
common_prompt_common_message = '''Respond to the human as helpfully and accurately as possible. 
    
You have access to the following tools: {tools}

Valid "action" values: "Final Answer" or {tool_names}

To use a tool, you MUST respond with a JSON object in the following format:
{{ "action": "tool_name", "action_input": "tool_input" }}

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
Observation: action result
... (repeat Thought/Action/Observation N times)
When action result contains the phrase 'Final Answer', do not repeat further.

Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}

Human: {input}
Assistant: 

{agent_scratchpad}

'''

In [5]:
class AgentFactory:
    def __init__(self, llm):
        self.llm = llm

    def create_flight_agent(self):
        tools = [TravelTools.search_flights()]
        prompt_txt = 'You are an AI flight search assistant. '+ common_prompt_common_message
        prompt = ChatPromptTemplate.from_template(prompt_txt)
        agent = create_structured_chat_agent(self.llm, tools, prompt)
        return AgentExecutor(agent=agent, 
                             tools=tools, 
                             verbose=True, 
                             handle_parsing_errors=True)

    def create_train_agent(self):
        tools = [TravelTools.search_trains()]
        prompt_txt = 'You are an AI train search assistant. '+ common_prompt_common_message
        prompt = ChatPromptTemplate.from_template(prompt_txt)
        agent = create_structured_chat_agent(self.llm, tools, prompt)
        return AgentExecutor(agent=agent, 
                             tools=tools, 
                             verbose=True, 
                             handle_parsing_errors=True)

    def create_travel_assistant_agent(self):
        tools = [TravelTools.search_hotels()]
        prompt_txt = 'You are an AI hotel search assistant. '+ common_prompt_common_message
        prompt = ChatPromptTemplate.from_template(prompt_txt)
        agent = create_structured_chat_agent(self.llm, tools, prompt)
        return AgentExecutor(agent=agent, 
                             tools=tools, 
                             verbose=True, 
                             handle_parsing_errors=True)

    def create_stock_agent(self):
        tools = [FinanceTools.search_stocks()]
        prompt_txt = 'You are an AI stock search assistant. '+ common_prompt_common_message
        prompt = ChatPromptTemplate.from_template(prompt_txt)
        agent = create_structured_chat_agent(self.llm, tools, prompt)
        return AgentExecutor(agent=agent, 
                             tools=tools, 
                             verbose=True, 
                             handle_parsing_errors=True)

    def create_mutual_fund_agent(self):
        tools = [FinanceTools.search_mutual_funds()]
        prompt_txt = 'You are an AI mutual fund search assistant. '+ common_prompt_common_message
        prompt = ChatPromptTemplate.from_template(prompt_txt)
        agent = create_structured_chat_agent(self.llm, tools, prompt)
        return AgentExecutor(agent=agent, 
                             tools=tools, 
                             verbose=True, 
                             handle_parsing_errors=True)

    def create_finance_assistant_agent(self):
        tools = [FinanceTools.get_financial_advice()]
        prompt_txt = 'You are an AI  financial advisor assistant. '+ common_prompt_common_message
        prompt = ChatPromptTemplate.from_template(prompt_txt)
        agent = create_structured_chat_agent(self.llm, tools, prompt)
        return AgentExecutor(agent=agent, 
                             tools=tools, 
                             verbose=True, 
                             handle_parsing_errors=True)

In [6]:
class SupervisorSystem:
    def __init__(self):
        self.llm = AzureChatOpenAI(
            azure_deployment="gpt-4",  # or your deployment
            api_version=os.environ.get('AZURE_API_VERSION'),
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2,
            )   #ChatOpenAI(api_key=api_key)
        self.agent_factory = AgentFactory(self.llm)
        
        # Initialize all agents
        self.flight_agent = self.agent_factory.create_flight_agent()
        self.train_agent = self.agent_factory.create_train_agent()
        self.travel_assistant = self.agent_factory.create_travel_assistant_agent()
        self.stock_agent = self.agent_factory.create_stock_agent()
        self.mutual_fund_agent = self.agent_factory.create_mutual_fund_agent()
        self.finance_assistant = self.agent_factory.create_finance_assistant_agent()

    def _route_travel_query(self, query: str):
        # Route to appropriate travel agent based on keywords
        query_lower = query.lower()
        if "flight" in query_lower or "airline" in query_lower or "plane" in query_lower:
            return self.flight_agent
        elif "train" in query_lower or "railway" in query_lower:
            return self.train_agent
        else:
            return self.travel_assistant

    def _route_finance_query(self, query: str):
        # Route to appropriate finance agent based on keywords
        query_lower = query.lower()
        if "stock" in query_lower or "shares" in query_lower:
            return self.stock_agent
        elif "mutual fund" in query_lower or "mf" in query_lower:
            return self.mutual_fund_agent
        else:
            return self.finance_assistant

    async def process_query(self, query: str) -> str:
        # Determine query type and route to appropriate supervisor
        travel_keywords = {"travel", "flight", "train", "hotel", "booking", "trip", "vacation"}
        finance_keywords = {"money", "invest", "stock", "fund", "financial", "market"}
        
        query_lower = query.lower()
        is_travel = any(keyword in query_lower for keyword in travel_keywords)
        is_finance = any(keyword in query_lower for keyword in finance_keywords)

        try:
            if is_travel:
                agent = self._route_travel_query(query)
            elif is_finance:
                agent = self._route_finance_query(query)
            else:
                # Default to travel assistant for unclassified queries
                agent = self.travel_assistant

            tool_name = agent.tools[0].name
            print("tool_name = "+tool_name)
            # Execute the query using the selected agent
            response = await agent.ainvoke({"input": query, "tool_names": [tool_name]})
            return response
        except Exception as e:
            return f"Error processing query: {str(e)}"

In [8]:
# Initialize the system
system = SupervisorSystem()

# Example queries
example_queries = [
    "I need to book a flight from New York to London",
    "What are the best mutual funds to invest in?",
    "Help me plan a vacation in Europe",
    "Should I invest in tech stocks?",
    "I need a train ticket to Boston"
]

# Process each query
for query in example_queries:
    print(f"\nProcessing query: {query}")
    result = await system.process_query(query)
    print(f"Response: {result}")


Processing query: I need to book a flight from New York to London
tool_name = search_flights


> Entering new AgentExecutor chain...
Thought: The user is requesting assistance in finding a flight from New York to London. I will use the search_flights tool to find available options.

Action:
```
{
  "action": "search_flights",
  "action_input": {
    "tool_input": "New York to London"
  }
}
Thought: Since the observation indicates a "Final Answer" with flight search results, I will provide this information to the user.

Action:
```
{
  "action": "Final Answer",
  "action_input": "I have found a list of available flights from New York to London. Please let me know if you need further assistance with booking or if you have specific preferences for your flight."
}
```

> Finished chain.
Response: {'input': 'I need to book a flight from New York to London', 'tool_names': ['search_flights'], 'output': 'I have found a list of available flights from New York to London. Please let me know if y